In [2]:
!unzip "/content/trainingandtestdata.zip"

Archive:  /content/trainingandtestdata.zip
replace testdata.manual.2009.06.14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testdata.manual.2009.06.14.csv  
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: training.1600000.processed.noemoticon.csv  y



In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.pipeline import make_pipeline
import os, random,io
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:
name_col = ["Polarity","id","date","query","user","text"]

Complete_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv",names=name_col , encoding='latin-1')

In [5]:
positive_set = Complete_data[Complete_data["Polarity"] == 0 ]
neagtive_set = Complete_data[Complete_data["Polarity"] == 4 ]

PN_Combined = pd.concat([positive_set,neagtive_set])

In [6]:
new_col = ["Polarity","text"]
PN_Clean = pd.DataFrame(data=PN_Combined[new_col])

In [8]:
PN_Clean["Polarity"]=PN_Clean["Polarity"].replace(4,1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

1. Take the positive and the negative tweets only. Use Sklearn to split the dataset in 80% training, 20% testing splits. Provide a nicely formatted summary of these splits, containing their size) (15 points)



In [17]:
x_train,x_test,y_train,y_test = train_test_split(PN_Clean.text,PN_Clean.Polarity,test_size=0.2,random_state=2361)
#Train_data,Testing_data = train_test_split(PN_Clean,test_size=0.20,random_state=2361)

In [79]:
print("The size of training data set for tweet text is",x_train.size)
print("The size of testing data set for tweet text is",x_test.size)

print("The size of training data set for tweet Polarity is",y_train.size)
print("The size of testing data set for tweet Polarity is",y_test.size)

The size of training data set for tweet text is 1280000
The size of testing data set for tweet text is 320000
The size of training data set for tweet Polarity is 1280000
The size of testing data set for tweet Polarity is 320000


2. Use the code from the previous classes to build the following models (15 points): 



a) SVM using TF-IDF.

In [ ]:
SVM_model = make_pipeline(TfidfVectorizer(),LinearSVC()) 
SVM_model = RF_model.fit(x_train,y_train)

b) NaiveBayes using TF-IDF.

In [20]:
NV_model = make_pipeline(TfidfVectorizer(),MultinomialNB()) 
NV_model = RF_model.fit(x_train,y_train)

c) Random Forest using TF-IDF.

In [21]:
RF_model = make_pipeline(TfidfVectorizer(),RandomForestClassifier(max_depth=20)) 
RF_model = RF_model.fit(x_train,y_train)

3. Use the code from the LSTM class to build a classifier for negative and positive sentiment tweets. Train the model with the training data split. Once the model is built, test it with the testing data split. Display the classifier report for this evaluation. Answer the following question: What can you say about the performance of this model? (40 points)

In [24]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf 
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
import logging

In [56]:
def LSTM_Classify(PN_Clean,x_train,x_test,y_train,y_test):

  print("Building a LSTM Classification Model")
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(PN_Clean.text)

  word_index = tokenizer.word_index
  vocad_size = len(tokenizer.word_index) + 1

  result = [len(x) for x in PN_Clean.text]

  review_length = len(max((PN_Clean.text), key=len))

  x_train = pad_sequences(tokenizer.texts_to_sequences(x_train),maxlen = review_length)
  x_test = pad_sequences(tokenizer.texts_to_sequences(x_test),maxlen = review_length)

  print("Shape Training Review Data: " + str(x_train.shape))
  print("Shape Training Class Data: " + str(x_test.shape))

  model = tf.keras.models.Sequential()

  model.add(
      tf.keras.layers.Embedding(
          input_dim = vocad_size, # The size of our vocabulary 
          output_dim = 32, # Dimensions to which each words shall be mapped
          input_length = review_length # Length of input sequences
      )
  )

  model.add(
      tf.keras.layers.Dropout(
          rate=0.25 # Randomly disable 25% of neurons
      )
  )

  model.add(
      tf.keras.layers.LSTM(
          units=32 # 32 LSTM units in this layer
      )
  )

  model.add(
      tf.keras.layers.Dropout(
          rate=0.25 # Randomly disable 25% of neurons
      )
  )


  model.add(
      tf.keras.layers.Dense(
          units=1, # Single unit
          activation='sigmoid' # Sigmoid activation function (output from 0 to 1)
      )
  )

  # Compile the model
  model.compile(
      loss=tf.keras.losses.binary_crossentropy, # loss function
      optimizer=tf.keras.optimizers.Adam(), # optimiser function
      metrics=['accuracy']) # reporting metric

  # Display a summary of the models structure
  model.summary()

  history = model.fit(
    x_train, y_train,           
    batch_size=256, 
    epochs=3, 
    validation_split=0.2,
    verbose=1
  ) 


  return history


In [ ]:
LSTM_Model_1 = LSTM_Classify(PN_Clean,x_train,x_test,y_train,y_test)

In [ ]:
from sklearn.metrics import classification_report
predicted_classes = model.predict_classes(x_test)
print(classification_report(y_test, predicted_classes, target_names=class_names))

4. Compare all models together in terms of Precision, Recall and F1 score. Put all of these numbers in a nicely formatted dataframe. Answer the following questions: Which model performs the best? Why do you think this is? What do you think you can do to improve performance? (30 points)

In [37]:
x_train,x_test,y_train,y_test = train_test_split(PN_Clean.text,PN_Clean.Polarity,test_size=0.2,random_state=2361)

In [38]:
label_1=NV_model.predict(x_test)
label_2=RF_model.predict(x_test)

In [ ]:
label_3=SVM_model.predict(x_test)

In [50]:
f1Score1 = []
precision1 = []
recall1 = []

columns=['Model_Name','Vectorizer','Precision','Recall','Macro_F1-score']
Modelnames = ['Naive_Bayes','RandomForest','Support_Vector_Machines']

def scorefunction(label,testdata):
  temp= sklearn.metrics.f1_score(label,y_test,average='macro')
  temp3 = sklearn.metrics.precision_score(y_test,label)
  temp4 = sklearn.metrics.recall_score(y_test,label)

  f1Score1.append(temp)
  precision1.append(temp3)
  recall1.append(temp4)

In [51]:
import sklearn
scorefunction(label_1,y_test)
scorefunction(label_2,y_test)
scorefunction(label_3,y_test)

In [52]:
columns=['Model_Name','Precision','Recall','Macro_F1-score']
performace = pd.DataFrame()
performace['Model_Name'] = Modelnames
performace['Precision'] = precision1
performace['Recall'] = recall1
performace['Macro_F1-score'] = f1Score1

5. Add to the comparison of #4 a the manually calculated precision, recall and F1 score using VADER and their suggested defaults to categorize the test split tweets in positive or negative. Answer the following questions: Is this approach as good as the previous ones? Why do you think this is? (30 points)

In [61]:
import nltk 
import numpy
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [67]:
def VADER_STUFF(x_test):

    new_copy = pd.DataFrame(x_test,columns=["text"])
    compound = []

    for i in new_copy['text']:
      generated_array = SentimentIntensityAnalyzer().polarity_scores(i)
      compound.append(generated_array["compound"])

    new_copy["compound"] = compound

    value = []
    for k in new_copy["compound"]:
      if row >= 0.05:
          value.append("1")
      else:
        value.append("0")

In [ ]:
VADER_STUFF(x_test)

In [ ]:
Vad_pred = new_copy.value

accuracy5 = accuracy_score(y_test,Vad_pred)
recall5 = recall_score(y_test,Vad_pred)
precision5 = precision_score(y_test,Vad_pred)
f1 =() 2 * (precision5*recall5)) /(precision5+recall5)

Print("Using VADER our Accuracy score is "+accuracy5)
Print("Recall is "+recall5)
Prrint("Precison is "+precision5)

In [ ]:
Print("Using VADER our Accuracy score is "+accuracy5)
Print("Recall is "+recall5)
Prrint("Precison is "+precision5)

Bonus (30 points): Try the following things to improve the LSTM model:

Answer the following questions: Did this change the results in any way? Why do you think so?

**THESE DIDNT COMPILE ON TIME.PLEASE CHECK MY GITHUB FOR THE OUTPUT CELLS.THANK YOU**

[MY GITHUB PAGE](https://https://github.com/Kausik-A/CSC-8980-Natural-Language-Processing)

1) Use 90% training data, 10% testing

In [ ]:
x_train_2,x_test_2,y_train_2,y_test_2 = train_test_split(PN_Clean.text,PN_Clean.Polarity,test_size=0.2,random_state=2361)
LSTM_Model_2 = LSTM_Classify(PN_Clean,x_train_2,x_test_2,y_train_2,y_test_2)

2) Remove stopwords from the tweets.

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords  

nltk.download('punkt')
nltk.download('stopwords')


stop_words = set(stopwords.words('english'))
new_copy_5 = PN_Clean.copy()

LSTM_Model_2 = LSTM_Classify(new_copy_5,x_train_2,x_test_2,y_train_2,y_test_2)

3) Remove all user mentions for the tweets (@something)
Compare all three new models in terms of their precision, recall and F1 score. 